In [42]:
import requests
from pprint import pp 
from statistics import mean
from datetime import datetime 

In [2]:
# find the max number of items
def get_max_items():
    url = ' https://hacker-news.firebaseio.com/v0/maxitem.json?print=pretty'
    data = requests.get(url)
    response = data.json()

    return response 

get_max_items()

34347114

In [ ]:
# get the last 1000 items
def get_items():
    url = 'https://hacker-news.firebaseio.com/v0/item/'

    items = []
    for i in range(get_max_items()-999, get_max_items()+1):
        data = requests.get(url+str(i)+'.json')
        response = data.json()
        
        items.append(response)

    return items 

all_items = get_items()
all_items

In [28]:
# Write a function to display all the items written by a certain user
def get_user_items(user_name):
    all_user_items = []
    for item in all_items:
        try:
            if item['by'] == user_name:
                all_user_items.append(item)
        except:
            continue 
        
    return all_user_items        
        
get_user_items('borroka')

[{'by': 'borroka',
  'id': 34346950,
  'parent': 34346860,
  'text': '&gt; They&#x27;re not &quot;Veeps&quot;, they&#x27;re people who have an identity and existence without reference to their roles at Verily<p>The words used by HR from this and other companies clearly show that for the company, they are not people with an identity, existence or raison d&#x27;être other than providing some kind of service to the company. The invitation is to treat them accordingly.',
  'time': 1673481666,
  'type': 'comment'},
 {'by': 'borroka',
  'id': 34347054,
  'parent': 34346824,
  'text': '&gt; I think that&#x27;s probably a natural tendency for any type of people...<p>I agree about the natural tendency, the call of the &quot;tribe&quot; (ethnic group, common language, etc.) is there for all kinds of people, more or less.\nBut there is a difference, and I&#x27;m talking about frequency distributions of behaviors, not naive black and white: some groups of people are aware of these tendencies, and 

In [32]:
# Write a function to display the average score of all items
def avg_score_items():
    scores = []
    for item in all_items:
        try:
            scores.append(item['score'])
        except:
            continue
    
    # print(scores)
    avg = mean(scores)
    return avg

avg_score_items()

4.083333333333333

In [8]:
# Write a function to display the average score of the items by a certain user
def avg_score_user(user_name):
    item_ids = get_user_items(user_name)
    
    scores = []
    for item in item_ids:
        try:
            scores.append(item['score'])
        except:
            continue 

    if len(scores) == 0: 
        return 0
    return mean(scores)

avg_score_user('erehweb')

3

In [39]:
# Write a function to display all of the comments of an item(this is a more challenging task)
# only did 1 level up and 1 level down
# need to use recursion in order to get all levels
def get_item(id):
    for item in all_items:
        if item['id'] == id:
            return item
#----------------------------------------------------
def get_comments(id):
    item = get_item(id)
    
    all_comments = []
    try:
        all_comments.append(item['text'])
        all_comments.append(get_item(item['parent'])['text'])

        for kid in item['kids']:
            kid_item = get_item(kid)
            all_comments.append(kid_item['text'])
    except:
        pass 
    
    return all_comments
    
all_comments = get_comments(34347059)
print(len(all_comments), 'comments')
all_comments

3 comments


['Like I said, it&#x27;s not a good title. It will get fixed.',
 'Yeah yeah, but &quot;an Google&quot;?',
 'It’s also breaking the guideline against editorialising in the title']

In [ ]:
# Write a function to display all of the items created in a certain year. 
# You can expand on this to add additional parameters: Month, Day, etc. 
# Any combo you choose or any single value
from datetime import datetime 

def items_per_year(datepart, value):
    items = []

    for item in all_items:
        if datepart == 'year':
            item_year = datetime.fromtimestamp(item['time']).strftime('%Y')
            if int(item_year) == value:
                items.append(item)
        
        if datepart == 'month':
            item_month = datetime.fromtimestamp(item['time']).strftime('%B')
            if item_month == value:
                items.append(item)

        if datepart == 'day':
            item_day = datetime.fromtimestamp(item['time']).strftime('%A')
            if item_day == value:
                items.append(item)

    return items 

# items_per_year('year', 2023)
# items_per_year('month', 'January')
items_per_year('day', 'Wednesday')

In [24]:
# Write a function to display the total number of deleted items.
def deleted_items():
    count = 0

    for item in all_items:
        try:
            if item['deleted'] == True:
                count += 1
        except:
            continue 

    return count 

deleted_items()

32

In [25]:
# Write a function to display the total number of live items.
def live_items():
    count = 0

    for item in all_items:
        try:
            if item['dead'] == True:
                count += 1
        except:
            continue 

    return len(all_items) - count 

live_items()

974

In [37]:
# Write a function to find which item has the most total comments
def most_comments():
    max_count = 0
    max_item = {}

    for item in all_items:
        try:
            all_comments = get_comments(item['id'])
            
            if len(all_comments) > max_count:
                max_count = len(all_comments)
                max_item = item
        except:
            continue

    return max_item

most_comments()        

{'by': 'idopmstuff',
 'id': 34346486,
 'kids': [34346589,
  34346868,
  34346501,
  34346789,
  34346521,
  34346603,
  34346989],
 'parent': 34346218,
 'text': 'As far as I can tell, layoffs seem concentrated among high-earning folks - I can definitely see how anyone browsing HN would think we&#x27;re headed straight for a recession. But it seems like overall, and especially in lower-wage jobs, employment is still humming along and people are very much not getting laid off.<p>That actually really gives me hope for a soft landing - those high-wage folks are much less likely to have serious financial problems like homes getting foreclosed (not to say it&#x27;s not possible but they&#x27;re much more likely to have a cushion to keep making payments). If they pull back on their discretionary spending plus stop doing things like driving up housing&#x2F;car&#x2F;stock prices by putting money into those things, that feels like it could be the basis for inflation slowing down without a huge u

In [40]:
# Write a function to find which user made the most comments.
def user_with_most_comments():
    max_count = 0
    max_user = {}
    
    for item in all_items:
        try:
            count = len(get_user_items(item['by']))

            if count > max_count:
                max_count = count 
                max_user = item 
        except:
            continue 

    return max_user['by']  

user_with_most_comments()

'TeMPOraL'

In [43]:
# Write a function to see which hour of the day is the most common for a user to post an item
def hour_of_day():
    results = {}
    for item in all_items:
        item_hour = datetime.fromtimestamp(item['time']).strftime('%H')
        
        if item_hour in results:
            results[item_hour] += 1
        else:
            results[item_hour] = 1

    max_count = max(results.values())

    for item in results:
        if results[item] == max_count:
            return f'{item}:00'

hour_of_day()

'18:00'